In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np

## 2014 Ebola Outbreak Data

[Caitlin Rivers' `ebola` GitHub repository](https://github.com/cmrivers/ebola) contains summarized reports of Ebola cases from three countries during the recent outbreak of the disease in West Africa. These data are licenced for both commercial and non-commercial use. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

![ebola](images/ebola.jpg)

From these data files, we will use pandas to import them and create a single data frame that includes the **daily totals of new cases** for each country. 

In [ ]:
url_base = 'https://raw.githubusercontent.com/cmrivers/ebola/master/guinea_data/'

In [ ]:
pd.read_csv(url_base+'2014-09-02.csv').head()

Clearly, we will need to develop row **masks** to extract the data we need across all files, without having to manually extract data from each file.

Let's hack at one file to develop the mask.

In [ ]:
sample = pd.read_csv(url_base+'2014-09-02.csv')

To prevent issues with capitalization, we will simply revert all labels to lower case.

In [ ]:
lower_vars = sample.Description.str.lower()

Since we are interested in extracting new cases only, we can use the **string accessor** attribute to look for key words that we would like to include or exclude.

In [ ]:
case_mask = (lower_vars.str.contains('new') 
             & (lower_vars.str.contains('case') | lower_vars.str.contains('suspect')) 
             & ~lower_vars.str.contains('non')
             & ~lower_vars.str.contains('total'))

We could have instead used regular expressions to do the same thing.

Finally, we are only interested in three columns.

In [ ]:
sample.loc[case_mask, ['Date', 'Description', 'Totals']]

We can now embed this operation in a loop over all the filenames in the database. We first need to create a range of dates, since the data files are organized by date.

In [ ]:
outbreak_dates = [str(date).split(' ')[0] for date in pd.date_range('2014-07-01', '2014-11-30')]

In [ ]:
datasets = []
for country in ('guinea', 'sl', 'liberia'):
    
    print('Getting {} data files'.format(country))
    for date in outbreak_dates:
        
        file = 'https://raw.githubusercontent.com/cmrivers/ebola/master/{}_data/{}.csv'.format(country, date)
        try:
            data = pd.read_csv(file)
            print('\t{}'.format(date))
        except Exception:
            continue
        
        # Convert to lower case to avoid capitalization issues
        data.columns = data.columns.str.lower()
        # Column naming is inconsistent. These procedures deal with that.
        keep_columns = ['date']
        if 'description' in data.columns:
            keep_columns.append('description')
        else:
            keep_columns.append('variable')
            
        if 'totals' in data.columns:
            keep_columns.append('totals')
        else:
            keep_columns.append('national')
            
        # Index out the columns we need, and rename them
        keep_data = data[keep_columns]
        keep_data.columns = 'date', 'variable', 'totals'
        
        # Extract the rows we might want
        lower_vars = keep_data.variable.str.lower()
        # Of course we can also use regex to do this
        case_mask = (lower_vars.str.contains('new') 
                     & (lower_vars.str.contains('case') | lower_vars.str.contains('suspect') 
                                                        | lower_vars.str.contains('confirm')) 
                     & ~lower_vars.str.contains('non')
                     & ~lower_vars.str.contains('total'))
        
        keep_data = keep_data[case_mask].dropna()
        
        # Convert data types
        keep_data['date'] = pd.to_datetime(keep_data.date)
        keep_data['totals'] = keep_data.totals.astype(int)
        
        # Assign country label and append to datasets list
        datasets.append(keep_data.assign(country=country))
        

Now that we have a list populated with `DataFrame` objects for each day and country, we can call `concat` to concatenate them into a single `DataFrame`.

In [ ]:
all_data = pd.concat(datasets)
all_data.head()

This works because the structure of each table was identical

### Manipulating indices

Notice from above, however, that the index contains redundant integer index values. We can confirm this:

In [ ]:
all_data.index.is_unique

We can create a new unique index by calling the `reset_index` method on the new data frame after we import it, which will generate a new ordered, unique index.

In [ ]:
all_data = pd.concat(datasets).reset_index(drop=True)
all_data.head()

**Reindexing** allows users to manipulate the data labels in a DataFrame. It forces a DataFrame to conform to the new index, and optionally, fill in missing data if requested.

A simple use of `reindex` is to alter the order of the rows. For example, records are currently ordered first by country then by day, since this is the order in which they were iterated over and imported. We might arbitrarily want to reverse the order, which is performed by passing the appropriate index values to `reindex`.

In [ ]:
all_data.reindex(all_data.index[::-1])

Notice that the reindexing operation is not performed "in-place"; the original `DataFrame` remains as it was, and the method returns a copy of the `DataFrame` with the new index. This is a common trait for pandas, and is a Good Thing.

We may also wish to reorder the columns this way.

In [ ]:
all_data.reindex(columns=['date', 'country', 'variable', 'totals']).head()

## Group by operations

One of pandas' most powerful features is the ability to perform operations on subgroups of a `DataFrame`. These so-called **group by** operations defines subunits of the dataset according to the values of one or more variabes in the `DataFrame`.

For this data, we want to sum the new case counts by day and country; so we pass these two column names to the `groupby` method, then sum the `totals` column accross them.

In [ ]:
all_data_grouped = all_data.groupby(['country', 'date'])
daily_cases = all_data_grouped['totals'].sum()
daily_cases.head(10)

The resulting series retains a hierarchical index from the group by operation. Hence, we can index out the counts for a given country on a particular day by indexing with the appropriate tuple.

In [ ]:
daily_cases[('liberia', '2014-09-02')]

One issue with the data we have extracted is that there appear to be serious **outliers** in the Liberian counts. The values are much too large to be a daily count, even during a serious outbreak.

In [ ]:
daily_cases.sort_values(ascending=False)
daily_cases.head(10)

We can filter these outliers using an appropriate threshold.

In [ ]:
daily_cases = daily_cases[daily_cases<200]

## Plotting

pandas data structures have high-level methods for creating a variety of plots, which tends to be easier than generating the corresponding plot using matplotlib. 

For example, we may want to create a plot of the cumulative cases for each of the three countries. The easiest way to do this is to remove the hierarchical index, and create a `DataFrame` of three columns, which will result in three lines when plotted.

First, call `unstack` to remove the hierarichical index:

In [ ]:
daily_cases.unstack().head()

Next, transpose the resulting `DataFrame` to swap the rows and columns.

In [ ]:
daily_cases.unstack().T.head()

Since we have missing values for some dates, we will assume that the counts for those days were zero (the actual counts for that day may have bee included in the next reporting day's data).

In [ ]:
daily_cases.unstack().T.fillna(0).head()

Finally, calculate the cumulative sum for all the columns, and generate a line plot, which we get by default.

In [ ]:
daily_cases.unstack().T.fillna(0).cumsum().plot()

## Resampling

An alternative to filling days without case reports with zeros is to aggregate the data at a coarser time scale. New cases are often reported by week; we can use the `resample` method to summarize the data into weekly values.

In [ ]:
weekly_cases = daily_cases.unstack().T.resample('W').sum()
weekly_cases

In [ ]:
weekly_cases.cumsum().plot()